In [1]:
from io import open
import unicodedata
import string
import re
import random
import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F
import warnings

warnings.filterwarnings("ignore")
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
SOS_token = 0
EOS_token = 1

class LanguageVocabulary(object):
    def __init__(self, name):
        self.name = name
        self.word2index = {}
        self.word2count = {}
        self.index2word = {0: "SOS", 1: "EOS"}
        self.n_words = 2

    def add_sentence(self, sentence):
        for word in sentence.split(' '):
            self.add_word(word)


    def add_word(self, word):
        if word not in self.word2index:
            self.word2index[word] = self.n_words
            self.word2count[word] = 1
            self.index2word[self.n_words] = word
            self.n_words += 1
        else:
            # Если такое уже слово есть просто добавляем 1 что добавилось одно слово
            self.word2count[word] += 1

In [4]:
def unicode_to_ascii(s):
    return ''.join(
        c for c in unicodedata.normalize('NFD', s)
        if unicodedata.category(c) != 'Mn'
    )

def normalize_string(s):
    s = unicode_to_ascii(s.lower().strip())
    # s = re.sub(r"([.!?])", r" \1", s)
    s = re.sub(r"[^a-zA-ZĄĆĘŁŃÓŚŹŻąćęłńóśźż.!?]+", r" ", s)
    return s

In [5]:
def read_languages(lang1, lang2, reverse=False):
    print("Reading lines...")
    lines = open('/content/drive/MyDrive/RNN/pol.txt', encoding='utf-8').read().strip().split('\n')
    pairs = [[normalize_string(s) for s in l.split('\t')[:2]] for l in lines]
    if reverse:
        pairs = [list(reversed(p)) for p in pairs]
        input_lang = LanguageVocabulary(lang2)
        output_lang = LanguageVocabulary(lang1)
    else:
        input_lang = LanguageVocabulary(lang1)
        output_lang = LanguageVocabulary(lang2)
    return input_lang, output_lang, pairs

In [6]:
def prepare_data(lang1, lang2, reverse=False):
    input_lang, output_lang, pairs = read_languages(lang1, lang2, reverse)
    print("Read %s sentence pairs" % len(pairs))
    print("Counting words...")
    for pair in pairs:
        input_lang.add_sentence(pair[0])
        output_lang.add_sentence(pair[1])
    print("Counted words:")
    print(input_lang.name, input_lang.n_words)
    print(output_lang.name, output_lang.n_words)
    return input_lang, output_lang, pairs

In [7]:
input_lang, output_lang, pairs = prepare_data('eng', 'pol', True)
print(random.choice(pairs))

Reading lines...
Read 46424 sentence pairs
Counting words...
Counted words:
pol 29928
eng 13324
['tom odpowiedział na pytania policjanta.', 'tom answered the policeman s questions.']


In [8]:
MAX_LENGTH = 100

In [9]:
class EncoderRNN(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(EncoderRNN, self).__init__()
        self.hidden_size = hidden_size
        self.embedding = nn.Embedding(input_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size)

    def forward(self, input, hidden):
        embedded = self.embedding(input).view(1, 1, -1)
        output = embedded
        output, hidden = self.gru(output, hidden)
        return output, hidden

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)

In [10]:
class DecoderRNN(nn.Module):
    def __init__(self, hidden_size, output_size):
        super(DecoderRNN, self).__init__()
        self.hidden_size = hidden_size
        self.embedding = nn.Embedding(output_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size)
        self.out = nn.Linear(hidden_size, output_size)
        self.softmax = nn.LogSoftmax(dim=1)

    def forward(self, input, hidden):
        output = self.embedding(input).view(1, 1, -1)
        output = F.relu(output)
        output, hidden = self.gru(output, hidden)
        output = self.softmax(self.out(output[0])) # берем output по нулевому индексу (одно предложение)
        return output, hidden

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)

In [11]:
def indexesFromSentence(lang, sentence):
    return [lang.word2index[word] for word in sentence.split(' ')]


def tensorFromSentence(lang, sentence):
    indexes = indexesFromSentence(lang, sentence)
    indexes.append(EOS_token)
    return torch.tensor(indexes, dtype=torch.long, device=device).view(-1, 1)

def tensorsFromPair(pair):
    input_tensor = tensorFromSentence(input_lang, pair[0])
    target_tensor = tensorFromSentence(output_lang, pair[1])
    return (input_tensor, target_tensor)

In [12]:
teacher_forcing_ratio = 0.5


def train(input_tensor, target_tensor, encoder, decoder, encoder_optimizer, decoder_optimizer, criterion, max_length=MAX_LENGTH):
    encoder_hidden = encoder.initHidden()
    encoder_optimizer.zero_grad()
    decoder_optimizer.zero_grad()
    input_length = input_tensor.size(0)
    target_length = target_tensor.size(0)
    encoder_outputs = torch.zeros(max_length, encoder.hidden_size, device=device)
    loss = 0
    for ei in range(input_length):
        encoder_output, encoder_hidden = encoder(input_tensor[ei], encoder_hidden)
        encoder_outputs[ei] = encoder_output[0, 0]


    decoder_input = torch.tensor([[SOS_token]], device=device)
    decoder_hidden = encoder_hidden

    use_teacher_forcing = True if random.random() < teacher_forcing_ratio else False
    if use_teacher_forcing:
        for di in range(target_length):
            decoder_output, decoder_hidden = decoder(decoder_input, decoder_hidden)
            loss += criterion(decoder_output, target_tensor[di])
            decoder_input = target_tensor[di]  # Teacher forcing
    else:
        for di in range(target_length):
            decoder_output, decoder_hidden = decoder(decoder_input, decoder_hidden)
            topv, topi = decoder_output.topk(1)
            decoder_input = topi.squeeze().detach()  # detach from history as input
            loss += criterion(decoder_output, target_tensor[di])
            if decoder_input.item() == EOS_token:
                break
    loss.backward()
    encoder_optimizer.step()
    decoder_optimizer.step()
    return loss.item() / target_length

In [13]:
import time
import math


def asMinutes(s):
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)


def timeSince(since, percent):
    now = time.time()
    s = now - since
    es = s / percent
    rs = es - s
    return '%s (- eta: %s)' % (asMinutes(s), asMinutes(rs))

In [19]:
def trainIters(encoder, decoder, n_iters, print_every=1000, plot_every=100, learning_rate=0.05):
    start = time.time()
    plot_losses = []
    print_loss_total = 0  # Reset every print_every
    plot_loss_total = 0  # Reset every plot_every

    encoder_optimizer = optim.SGD(encoder.parameters(), lr=learning_rate)
    decoder_optimizer = optim.SGD(decoder.parameters(), lr=learning_rate)
    training_pairs = [tensorsFromPair(random.choice(pairs)) for i in range(n_iters)]
    criterion = nn.NLLLoss()

    for epoch in range(1, n_iters):
        training_pair = training_pairs[epoch - 1]
        input_tensor = training_pair[0]
        target_tensor = training_pair[1]
        loss = train(input_tensor, target_tensor, encoder,
                     decoder, encoder_optimizer, decoder_optimizer, criterion)
        print_loss_total += loss
        plot_loss_total += loss

        if epoch % print_every == 0:
            print_loss_avg = print_loss_total / print_every
            print_loss_total = 0
            print('%s (%d %d%%) %.4f' % (timeSince(start, epoch / n_iters),
                                         epoch, epoch / n_iters * 100, print_loss_avg))

        if epoch % plot_every == 0:
            plot_loss_avg = plot_loss_total / plot_every
            plot_losses.append(plot_loss_avg)
            plot_loss_total = 0
    showPlot(plot_losses)

In [20]:
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
plt.switch_backend('agg')


def showPlot(points):
    plt.figure()
    fig, ax = plt.subplots()
    loc = ticker.MultipleLocator(base=0.2)
    ax.yaxis.set_major_locator(loc)
    plt.plot(points)

In [21]:
def evaluate(encoder, decoder, sentence, max_length=MAX_LENGTH):
    with torch.no_grad():
        input_tensor = tensorFromSentence(input_lang, sentence)
        input_length = input_tensor.size()[0]
        encoder_hidden = encoder.initHidden()
        encoder_outputs = torch.zeros(max_length, encoder.hidden_size, device=device)

        for i in range(input_length):
            encoder_output, encoder_hidden = encoder(input_tensor[i], encoder_hidden)
            encoder_outputs[i] += encoder_output[0, 0]

        decoder_input = torch.tensor([[SOS_token]], device=device)  # SOS
        decoder_hidden = encoder_hidden
        decoded_words = [] # Наши деокдированные слова

        for di in range(max_length):
            decoder_output, decoder_hidden = decoder(decoder_input, decoder_hidden)
            topv, topi = decoder_output.data.topk(1)
            if topi.item() == EOS_token:
                decoded_words.append('<EOS>')
                break
            else:
                decoded_words.append(output_lang.index2word[topi.item()])
            decoder_input = topi.squeeze().detach()
        return decoded_words

In [22]:
def evaluateRandomly(encoder, decoder, n=10):
    for i in range(n):
        pair = random.choice(pairs)
        print('>', pair[0])
        print('=', pair[1])
        output_words = evaluate(encoder, decoder, pair[0])
        output_sentence = ' '.join(output_words)
        print('<', output_sentence)
        print('')

In [23]:
hidden_size = 128
encoder1 = EncoderRNN(input_lang.n_words, hidden_size).to(device)
decoder1 = DecoderRNN(hidden_size, output_lang.n_words).to(device)
trainIters(encoder1, decoder1, 10*len(pairs), print_every=5000)

1m 13s (- eta: 112m 22s) (5000 1%) 4.8653
1m 57s (- eta: 89m 12s) (10000 2%) 4.5193
2m 42s (- eta: 81m 11s) (15000 3%) 4.4038
3m 28s (- eta: 77m 15s) (20000 4%) 4.2728
4m 15s (- eta: 74m 45s) (25000 5%) 4.2126
5m 0s (- eta: 72m 29s) (30000 6%) 4.0897
5m 45s (- eta: 70m 31s) (35000 7%) 4.0387
6m 30s (- eta: 69m 2s) (40000 8%) 3.9639
7m 15s (- eta: 67m 39s) (45000 9%) 3.9265
8m 1s (- eta: 66m 26s) (50000 10%) 3.9009
8m 46s (- eta: 65m 15s) (55000 11%) 3.8325
9m 31s (- eta: 64m 9s) (60000 12%) 3.7386
10m 16s (- eta: 63m 6s) (65000 14%) 3.7057
11m 1s (- eta: 62m 6s) (70000 15%) 3.6701
11m 46s (- eta: 61m 6s) (75000 16%) 3.6466
12m 32s (- eta: 60m 11s) (80000 17%) 3.6405
13m 17s (- eta: 59m 19s) (85000 18%) 3.5878
14m 2s (- eta: 58m 24s) (90000 19%) 3.5752
14m 48s (- eta: 57m 31s) (95000 20%) 3.5515
15m 32s (- eta: 56m 37s) (100000 21%) 3.4971
16m 17s (- eta: 55m 45s) (105000 22%) 3.4777
17m 3s (- eta: 54m 56s) (110000 23%) 3.4836
17m 48s (- eta: 54m 6s) (115000 24%) 3.3866
18m 34s (- eta: 

In [78]:
from torchtext.data.metrics import bleu_score
def evaluateRandomly(encoder, decoder, n=1000):
    candidate_corpus = []
    references_corpus = []
    for i in range(n):
        coded, origin = random.choice(pairs)
        references_corpus.append(origin.split(' '))
        output_words = evaluate(encoder, decoder, coded)
        candidate_corpus.append(output_words)
        output_sentence = ' '.join(output_words[:-1])
        if i%100 == 0:
          print('>', coded)
          print('=', origin)
          print('<', output_sentence, '  --> ', output_sentence.replace(' <EOS>', '') == origin)
          print('')
        # if output_sentence.replace(' <EOS>', '') == origin:
        #   acc +=1
    print(f'BLEU {bleu_score(candidate_corpus, references_corpus, max_n=2, weights = [0.5, 0.5])}')

In [79]:
evaluateRandomly(encoder1, decoder1)

> zrobiłem ciasto biszkoptowe.
= i made a sponge cake.
< i always wanted to my   -->  False

> nie przestawaj.
= don t stop.
< he didn   -->  False

> słyszałem jak spiewała.
= i heard her singing.
< i heard heard how to   -->  False

> skoncz kłamac.
= quit lying.
< he need   -->  False

> tom jest przeziebiony.
= tom has a cold.
< tom is   -->  False

> myslałem ze jasno powiedziałem ze nie chce tego robic.
= i thought i d made it clear that i didn t want to do that.
< i thought that didn t that that that that.   -->  False

> przyjdz tez jutro!
= come again tomorrow.
< the also a   -->  False

> nie mozesz jeszcze przejsc na emeryture.
= you can t retire yet.
< can can t go yet.   -->  False

> nigdy nie słyszałem jak mowi po angielsku.
= i have never heard him speak english.
< can speak speak never   -->  False

> ktora strona wygrała?
= which side won?
< he   -->  False

BLEU 0.006234269123524427
